## Spark assignment 2: Collocations
As for the second part of the assignment, your task is to extract collocations: that is word combinations that occur together. For example, “high school” or “roman empire”.

To find collocations, you will use NPMI (normalized pointwise mutual information) metric.

PMI of two words, a & b, is defined as

    “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”
   
where:
    - P(ab) is the probability of two words coming one after the other
    - P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is
    - “P(a) = # of occurrences of word a / total number of words”
    - “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

To build an intuition behind the definition, see Reading material.

Therefore, rare combinations of coupled words have large PMI.

NPMI is computed as 
    
    “NPMI(a, b) = PMI(a, b) / -ln P(ab)”. This normalizes the quantity to be within the range [-1; 1].

You task is a bit more complicated now:

- Extract all the words, as in the previous task.
- Filter out stopwords using the dictionary (/datasets/stop_words_en.txt ) (do not forget to convert words to the lowercase!)
- Compute all bigrams (that is, pairs of consequent words)
- Leave only bigrams with at least 500 occurrences
- Compute NPMI for every bigram (note: when computing probabilities, you need unpruned counts!)
- Sort word pairs by NPMI in the descending order
- Print top 39 word pairs, with words delimited by the underscore “_”

For example,

    roman_empire

    south_africa

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

### parse

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article).cache()

In [3]:
# result = wiki.take(1)[0]
# for word in result[:50]:
#     print word

In [4]:
stop_words_file = '/datasets/stop_words_en.txt'
def read_stop_words(file_path):
    return set(word.strip().lower() for word in open(file_path))
stop_words = read_stop_words(stop_words_file)

### words

In [5]:
words = wiki.flatMap(lambda x: [(x[i].lower(), 1) for i in range(len(x)-1)])
total_words = float(words.count())

In [6]:
words = words \
    .filter(lambda (x, y): x not in stop_words) \
    .reduceByKey(lambda x,y: x+y)

In [7]:
P_words = words.map(lambda (x, y): (x, float(y)/ total_words))

### pairs

In [8]:
pairs = wiki.flatMap(lambda x: [((x[i].lower(), x[i+1].lower()), 1) for i in range(len(x)-1)])
total_pairs = float(pairs.count())

In [9]:
pairs = pairs \
    .filter(lambda (x,y): (x[0] not in stop_words) and (x[1] not in stop_words)) \
    .reduceByKey(lambda x,y: x+y) \
    .filter(lambda (x,y): y >= 500)

In [10]:
P_pairs = pairs.map(lambda (x, y): (x, float(y)/ total_pairs))

### join

In [11]:
pairs = P_pairs.map(lambda ((a,b), P_ab): (a, (b, P_ab)))
pairs = pairs \
    .leftOuterJoin(P_words) \
    .map(lambda (a, ((b, P_ab), P_a)): (b, (a, P_a, P_ab))) \
    .leftOuterJoin(P_words) \
    .map(lambda (b, ((a, P_a, P_ab), P_b)): ((a, b), (P_a, P_b, P_ab)))

### NPMI

In [12]:
import numpy as np
PMI = pairs.map(lambda ((a, b), (P_a, P_b, P_ab)): ((a, b), (np.log(P_ab/(P_a*P_b)), P_ab)))
NPMI = PMI.map(lambda ((a, b), (PMI_ab, P_ab)): ((a, b), PMI_ab/np.log(P_ab)))
NPMI = NPMI.sortBy(lambda (x,y): y, ascending=False)

In [13]:
results = NPMI.take(39)
for result in results:
    print '%s_%s' % (result[0][0], result[0][1])

north_american
years_later
american_football
york_city
american_baseball
american_actress
american_actor
high_school
american_singer-songwriter
took_place
united_nations
roman_empire
south_africa
war_ii
catholic_church
world_war
civil_war
new_zealand
north_america
notes_references
roman_catholic
united_kingdom
university_press
baseball_player
air_force
soviet_union
references_external
20th_century
19th_century
supreme_court
new_york
et_al
san_francisco
prime_minister
united_states
external_links
los_angeles


## Test

In [14]:
%%writefile test.txt
minh tran phuong truong minh xuan thao phuong tran phuong cham phuong tran phuong xuan thao minh tam cham cham vy vy minh tran phuong cham cham cham cham

Overwriting test.txt


In [15]:
%%writefile stop_words.txt
minh
tam

Overwriting stop_words.txt


In [16]:
stop_words_file = 'stop_words.txt'
def read_stop_words(file_path):
    return set(word.strip().lower() for word in open(file_path))
stop_words = read_stop_words(stop_words_file)

### parse

In [17]:
import re

def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())
test = sc.textFile("test.txt", 16).map(normalizeWords).cache()
test.collect()

[[u'minh',
  u'tran',
  u'phuong',
  u'truong',
  u'minh',
  u'xuan',
  u'thao',
  u'phuong',
  u'tran',
  u'phuong',
  u'cham',
  u'phuong',
  u'tran',
  u'phuong',
  u'xuan',
  u'thao',
  u'minh',
  u'tam',
  u'cham',
  u'cham',
  u'vy',
  u'vy',
  u'minh',
  u'tran',
  u'phuong',
  u'cham',
  u'cham',
  u'cham',
  u'cham']]

### words

In [18]:
words = test.flatMap(lambda x: [(x[i].lower(), 1) for i in range(len(x)-1)]) 
words.collect()

[(u'minh', 1),
 (u'tran', 1),
 (u'phuong', 1),
 (u'truong', 1),
 (u'minh', 1),
 (u'xuan', 1),
 (u'thao', 1),
 (u'phuong', 1),
 (u'tran', 1),
 (u'phuong', 1),
 (u'cham', 1),
 (u'phuong', 1),
 (u'tran', 1),
 (u'phuong', 1),
 (u'xuan', 1),
 (u'thao', 1),
 (u'minh', 1),
 (u'tam', 1),
 (u'cham', 1),
 (u'cham', 1),
 (u'vy', 1),
 (u'vy', 1),
 (u'minh', 1),
 (u'tran', 1),
 (u'phuong', 1),
 (u'cham', 1),
 (u'cham', 1),
 (u'cham', 1)]

In [19]:
total_words = float(words.count())
total_words

28.0

In [20]:
words = words \
    .filter(lambda (x, y): x not in stop_words) \
    .reduceByKey(lambda x,y: x+y)
words.collect()

[(u'truong', 1),
 (u'thao', 2),
 (u'cham', 6),
 (u'xuan', 2),
 (u'tran', 4),
 (u'vy', 2),
 (u'phuong', 6)]

In [21]:
P_words = words.map(lambda (x, y): (x, float(y)/ total_words))
P_words.collect()

[(u'truong', 0.03571428571428571),
 (u'thao', 0.07142857142857142),
 (u'cham', 0.21428571428571427),
 (u'xuan', 0.07142857142857142),
 (u'tran', 0.14285714285714285),
 (u'vy', 0.07142857142857142),
 (u'phuong', 0.21428571428571427)]

### pairs

In [22]:
pairs = test.flatMap(lambda x: [((x[i].lower(), x[i+1].lower()), 1) for i in range(len(x)-1)])
pairs.collect()

[((u'minh', u'tran'), 1),
 ((u'tran', u'phuong'), 1),
 ((u'phuong', u'truong'), 1),
 ((u'truong', u'minh'), 1),
 ((u'minh', u'xuan'), 1),
 ((u'xuan', u'thao'), 1),
 ((u'thao', u'phuong'), 1),
 ((u'phuong', u'tran'), 1),
 ((u'tran', u'phuong'), 1),
 ((u'phuong', u'cham'), 1),
 ((u'cham', u'phuong'), 1),
 ((u'phuong', u'tran'), 1),
 ((u'tran', u'phuong'), 1),
 ((u'phuong', u'xuan'), 1),
 ((u'xuan', u'thao'), 1),
 ((u'thao', u'minh'), 1),
 ((u'minh', u'tam'), 1),
 ((u'tam', u'cham'), 1),
 ((u'cham', u'cham'), 1),
 ((u'cham', u'vy'), 1),
 ((u'vy', u'vy'), 1),
 ((u'vy', u'minh'), 1),
 ((u'minh', u'tran'), 1),
 ((u'tran', u'phuong'), 1),
 ((u'phuong', u'cham'), 1),
 ((u'cham', u'cham'), 1),
 ((u'cham', u'cham'), 1),
 ((u'cham', u'cham'), 1)]

In [23]:
total_pairs = float(pairs.count())
total_pairs

28.0

In [24]:
pairs = pairs \
    .filter(lambda (x,y): (x[0] not in stop_words) and (x[1] not in stop_words)) \
    .reduceByKey(lambda x,y: x+y) \
    .filter(lambda (x,y): y >= 2)
pairs.collect()

[((u'phuong', u'tran'), 2),
 ((u'tran', u'phuong'), 4),
 ((u'cham', u'cham'), 4),
 ((u'phuong', u'cham'), 2),
 ((u'xuan', u'thao'), 2)]

In [25]:
P_pairs = pairs.map(lambda (x, y): (x, float(y)/ total_pairs))
P_pairs.collect()

[((u'phuong', u'tran'), 0.07142857142857142),
 ((u'tran', u'phuong'), 0.14285714285714285),
 ((u'cham', u'cham'), 0.14285714285714285),
 ((u'phuong', u'cham'), 0.07142857142857142),
 ((u'xuan', u'thao'), 0.07142857142857142)]

### Join

In [26]:
pairs = P_pairs.map(lambda ((a,b), P_ab): (a, (b, P_ab)))
pairs.collect()

[(u'phuong', (u'tran', 0.07142857142857142)),
 (u'tran', (u'phuong', 0.14285714285714285)),
 (u'cham', (u'cham', 0.14285714285714285)),
 (u'phuong', (u'cham', 0.07142857142857142)),
 (u'xuan', (u'thao', 0.07142857142857142))]

In [27]:
pairs = pairs.leftOuterJoin(P_words)
pairs.collect()

[(u'tran', ((u'phuong', 0.14285714285714285), 0.14285714285714285)),
 (u'cham', ((u'cham', 0.14285714285714285), 0.21428571428571427)),
 (u'xuan', ((u'thao', 0.07142857142857142), 0.07142857142857142)),
 (u'phuong', ((u'tran', 0.07142857142857142), 0.21428571428571427)),
 (u'phuong', ((u'cham', 0.07142857142857142), 0.21428571428571427))]

In [28]:
pairs = pairs.map(lambda (a, ((b, P_ab), P_a)): (b, (a, P_a, P_ab)))
pairs.collect()

[(u'phuong', (u'tran', 0.14285714285714285, 0.14285714285714285)),
 (u'cham', (u'cham', 0.21428571428571427, 0.14285714285714285)),
 (u'thao', (u'xuan', 0.07142857142857142, 0.07142857142857142)),
 (u'tran', (u'phuong', 0.21428571428571427, 0.07142857142857142)),
 (u'cham', (u'phuong', 0.21428571428571427, 0.07142857142857142))]

In [29]:
pairs = pairs.leftOuterJoin(P_words)
pairs.collect()

[(u'thao',
  ((u'xuan', 0.07142857142857142, 0.07142857142857142), 0.07142857142857142)),
 (u'cham',
  ((u'cham', 0.21428571428571427, 0.14285714285714285), 0.21428571428571427)),
 (u'cham',
  ((u'phuong', 0.21428571428571427, 0.07142857142857142),
   0.21428571428571427)),
 (u'tran',
  ((u'phuong', 0.21428571428571427, 0.07142857142857142),
   0.14285714285714285)),
 (u'phuong',
  ((u'tran', 0.14285714285714285, 0.14285714285714285), 0.21428571428571427))]

In [30]:
pairs = pairs.map(lambda (b, ((a, P_a, P_ab), P_b)): ((a, b), (P_a, P_b, P_ab)))
pairs.collect()

[((u'xuan', u'thao'),
  (0.07142857142857142, 0.07142857142857142, 0.07142857142857142)),
 ((u'cham', u'cham'),
  (0.21428571428571427, 0.21428571428571427, 0.14285714285714285)),
 ((u'phuong', u'cham'),
  (0.21428571428571427, 0.21428571428571427, 0.07142857142857142)),
 ((u'phuong', u'tran'),
  (0.21428571428571427, 0.14285714285714285, 0.07142857142857142)),
 ((u'tran', u'phuong'),
  (0.14285714285714285, 0.21428571428571427, 0.14285714285714285))]

### NPMI

In [31]:
import numpy as np
PMI = pairs.map(lambda ((a, b), (P_a, P_b, P_ab)): ((a, b), (np.log(P_ab/(P_a*P_b)), P_ab)))
PMI.collect()

[((u'xuan', u'thao'), (2.6390573296152584, 0.07142857142857142)),
 ((u'cham', u'cham'), (1.1349799328389845, 0.14285714285714285)),
 ((u'phuong', u'cham'), (0.44183275227903923, 0.07142857142857142)),
 ((u'phuong', u'tran'), (0.84729786038720367, 0.07142857142857142)),
 ((u'tran', u'phuong'), (1.5404450409471491, 0.14285714285714285))]

In [32]:
NPMI = PMI.map(lambda ((a, b), (PMI_ab, P_ab)): ((a, b), PMI_ab/np.log(P_ab)))
NPMI.collect()

[((u'xuan', u'thao'), -0.99999999999999978),
 ((u'cham', u'cham'), -0.58326430610888502),
 ((u'phuong', u'cham'), -0.16742067226840154),
 ((u'phuong', u'tran'), -0.32106080109700724),
 ((u'tran', u'phuong'), -0.79163215305444257)]

In [33]:
NPMI = NPMI.sortBy(lambda (x,y): y, ascending=False)
NPMI.collect()

[((u'phuong', u'cham'), -0.16742067226840154),
 ((u'phuong', u'tran'), -0.32106080109700724),
 ((u'cham', u'cham'), -0.58326430610888502),
 ((u'tran', u'phuong'), -0.79163215305444257),
 ((u'xuan', u'thao'), -0.99999999999999978)]